## Imprter les module necessaires

In [1]:
import numpy as np
import pandas as pd
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, BatchNormalization
import pickle

2024-06-27 08:26:00.278735: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-27 08:26:00.362533: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-27 08:26:00.727969: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-27 08:26:03.065385: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import tensorflow as tf
print(tf.__version__)

def process_csv_files(folder_path):
    # Initialisation des listes
    data = []
    datalabel = []

    # Liste des fichiers dans le dossier
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Parcourir chaque fichier
    for file in files:
        # Chemin complet du fichier
        file_path = os.path.join(folder_path, file)

        # Lire le fichier CSV
        df = pd.read_csv(file_path)

        # Supprimer la colonne "label" si elle existe
        if 'label' in df.columns:
            df = df.drop(columns=['label'])

        # Diviser les données en segments de 50 lignes
        num_segments = len(df) // 50

        for i in range(num_segments):
            segment = df.iloc[i*50:(i+1)*50]
            data.append(segment.values.tolist())
            datalabel.append(file)

    # Sauvegarder les labels dans un fichier
    with open(os.path.join(folder_path, 'datalabel.txt'), 'w') as f:
        for label in datalabel:
            f.write(f"{label}\n")

    print("Traitement terminé.")
    return data, datalabel




2.16.1


In [ ]:
# Exemple d'utilisation
folder_path = "./backend"
data, datalabel = process_csv_files(folder_path)

## Fonction pour extraire les donnees dans un tenseurs et une matrice

In [ ]:
#methode pour extraire les donnees
def extration_data(path):
    data = []
    labels = []
    for subject_folder in os.listdir(path):
        subject_path = os.path.join(path, subject_folder)
        if os.path.isdir(subject_path):
            # Itérer sur chaque fichier dans le dossier (correspondant à chaque lettre ou mot)
            for file_name in os.listdir(subject_path):
                file_path = os.path.join(subject_path, file_name)
                if file_path.endswith('.csv'):
                    # Charger les données depuis le fichier CSV
                    df = pd.read_csv(file_path)
                    # Filtrer les colonnes requises
                    data_filtered = df[columns_to_keep]
                    # Diviser les données en segments de 150 échantillons chacun
                    num_samples_per_instance = 150
                    for start in range(0, len(data_filtered), num_samples_per_instance):
                        end = start + num_samples_per_instance
                        if end <= len(data_filtered):
                            segment = data_filtered.iloc[start:end].values
                            data.append(segment)
                            labels.append(file_name.split('.')[0])  # Utiliser le nom du fichier comme label
    return np.array(data), np.array(labels)

In [ ]:
# Chemin du répertoire contenant les dossiers de données
repertoire = "/content/drive/MyDrive/Colab Notebooks/data/ASL-Sensor-Dataglove-Dataset"

# Colonnes à conserver
columns_to_keep = ['flex_1', 'flex_2', 'flex_3', 'flex_4', 'flex_5', 'GYRx', 'GYRy', 'GYRz', 'ACCx', 'ACCy', 'ACCz']

# Appeler la fonction pour extraire les données
all_data, all_labels = extration_data(repertoire)

## Encoder les label pour les appreter a l'entrainement

In [ ]:
# Vectorize labels
label_encoder = LabelEncoder()
all_labels_encoded = label_encoder.fit_transform(all_labels)
all_labels_categorical = to_categorical(all_labels_encoded)

## Diviser le dataset en entrainement, validation et test

In [ ]:
# Ensure all labels are present in train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(all_data, all_labels_categorical, test_size=0.2, stratify=all_labels)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp)

print('train : X =>', X_train.shape,"y => ", y_train.shape)
print('val : X =>', X_val.shape,"y => ", y_val.shape)
print('test : X =>', X_test.shape,"y => ", y_test.shape)


train : X => (16800, 70, 11) y =>  (16800, 40)
val : X => (2100, 70, 11) y =>  (2100, 40)
test : X => (2100, 70, 11) y =>  (2100, 40)


## Construire le modele et la compiler pour initialiser certians parametres

In [ ]:
# Define the BiLSTM model
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(all_labels_categorical.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


### Entrainer le modele

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=34, validation_data=(X_val, y_val))

Epoch 1/30
495/495 [==============================] - 105s 194ms/step - loss: 1.8411 - accuracy: 0.4210 - val_loss: 0.8990 - val_accuracy: 0.6629
Epoch 2/30
495/495 [==============================] - 88s 178ms/step - loss: 0.9423 - accuracy: 0.6592 - val_loss: 0.7211 - val_accuracy: 0.7376
Epoch 3/30
135/495 [=======>......................] - ETA: 1:04 - loss: 0.7845 - accuracy: 0.7168

KeyboardInterrupt: 

##Tester/Evaluer le modele

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.4f}%")

32/32 [==============================] - 4s 139ms/step - loss: 0.1560 - accuracy: 0.9420
Test Accuracy: 94.2000%


## Suver le model et l'encodage dans des fichiers

In [ ]:
# Save the entire model
model.save('/content/drive/MyDrive/Colab Notebooks/data/bilstm_model.h5')
# Sauvegarde du LabelEncoder
with open('/content/drive/MyDrive/Colab Notebooks/data/label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Charger le modele et l'encodage pour une utilisation ulterieurs

In [ ]:
from tensorflow.keras.models import load_model
import pickle

# Charger le modèle
model_path = '/content/drive/MyDrive/Colab Notebooks/data/bilstm_model.h5'
model = load_model(model_path)

# Charger le LabelEncoder
label_encoder_path = '/content/drive/MyDrive/Colab Notebooks/data/label_encoder.pkl'
with open(label_encoder_path, 'rb') as file:
    label_encoder = pickle.load(file)


## Definir une fonction qui prend en entree un geste sous forme de data et predit le geste correspondant

In [ ]:

def load_and_predict_from_csv(file_path):
    # Charger les 150 premières lignes du CSV
    data = pd.read_csv(file_path, nrows=150)

    # Sélectionner les colonnes spécifiques
    columns_to_keep = ['flex_1', 'flex_2', 'flex_3', 'flex_4', 'flex_5', 'GYRx', 'GYRy', 'GYRz', 'ACCx', 'ACCy', 'ACCz']
    data_filtered = data[columns_to_keep]

    # Vérifier si nous avons bien 150 lignes, sinon compléter par des zéros
    if len(data_filtered) < 150:
        padding = np.zeros((150 - len(data_filtered), len(columns_to_keep)))
        data_filtered = np.vstack([data_filtered, padding])

    # Convertir en tenseur de dimension (1, pas_de_temps, feature)
    data_tensor = np.expand_dims(data_filtered, axis=0)

    # Faire la prédiction
    prediction = model.predict(data_tensor)

    # Trouver l'indice de la classe prédite
    predicted_class_index = np.argmax(prediction, axis=1)

    # Décoder l'indice en label
    predicted_label = label_encoder.inverse_transform(predicted_class_index)

    # Extraire le nom du fichier sans l'extension
    file_label = os.path.splitext(os.path.basename(file_path))[0]

    return file_label, predicted_label[0]


In [ ]:
# Exemple d'utilisation
file_path = '/content/drive/MyDrive/Colab Notebooks/data/ASL-Sensor-Dataglove-Dataset/001/thankyou.csv'
file_label, predicted_label = load_and_predict_from_csv(file_path)
print(f"Fichier: {file_label}, Geste prédit: {predicted_label}")

1/1 [==============================] - 0s 223ms/step
Fichier: thankyou, Geste prédit: thankyou
